In [1]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
import tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

store_list    = ["ricardoeletro", "magazineluiza"]

dataset_pages = f"data/heuristic_pages/*/*"
dataset_links = f"data/heusristic_links/heusristic_links/*"

In [4]:
dataset_data = [(url, pathlib.Path(url).stem.split('_')[0] ) for url in glob.glob(dataset_links) if len(url.split('_'))>3]

In [5]:
datasets = []

dataset_idx = 0
data_raw_s = pd.read_csv(dataset_data[dataset_idx][0])
data_raw_s = data_raw_s.dropna().drop_duplicates()
data_raw_s['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw_s.shape[0])

for dataset_idx in range(1, len(dataset_data)):
    data_raw = pd.read_csv(dataset_data[dataset_idx][0])
    data_raw = data_raw.dropna().drop_duplicates()
    data_raw['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw.shape[0])
    data_raw_s = pd.merge(data_raw_s, data_raw, how='outer')
    #datasets.append(data_raw)

In [19]:
x_pages = [f'data/heuristic_pages/{store}/page_{p_id}.html' for (store, p_id) in zip(data_raw_s['store'], data_raw_s['id'])]
y = data_raw_s['label'].values
u_class = data_raw_s['store']

In [22]:
X, idx_arr = pages_to_matrix(x_pages)
y = y[idx_arr]
u_class = u_class[idx_arr]
X.shape, y.shape, u_class.shape


Invalid Links 4


In [42]:
df = pd.DataFrame()
df['text'] = X
df['store'] = u_class
df['label'] = y
df.to_csv("text_store_label.csv", index=False)

In [12]:
X_save = X.copy()
y_save = y.copy()

In [8]:
head_text

['Amazon.com.br:Avaliação de clientes: Smartphone Xiaomi Redmi Note 7 64GB 4GB RAM Preto']

In [4]:
pages_to_matrix([s])

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: expected str, bytes or os.PathLike object, not BeautifulSoup

In [18]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [15]:
text_clf.fit(X_train, y_train)  
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.9630314232902033